In [1]:
import sys
import os

# Agrega la carpeta raíz del proyecto (src) al path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [3]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import datetime, timedelta
from meteostat import Point, Daily
from utils.utils import *
import requests

In [4]:
df_shark = pd.read_csv(r'..\data\shark_attack_clean.csv')
df_shark.drop(['Unnamed: 0', 'age'], axis=1, inplace=True)


In [5]:
import pandas as pd

# Cargar archivo
df = pd.read_csv(r"..\data\temperatura_mar.csv", sep='\s+', header=0)

# Opcional: limitar a años razonables
df = df[df["year"].between(1900, 2022)]


In [ ]:
df.head()

,year,lat,lon,count,value
8777,1900,12.5,107.5,5,0.19
8778,1900,12.5,122.5,1,-1.12
8779,1900,-12.5,132.5,2,-0.03
8780,1900,-12.5,-172.5,2,0.23
8781,1900,12.5,-17.5,2,-1.26


In [ ]:
# Nueva columna con las decadas, para utilizar esta columna en los gráficos
df_shark['decade'] = (df_shark['year'] // 10 * 10).astype(int)

In [ ]:
# Paso la columna date  a datetime y la columna year a int 
df_shark['date'] = pd.to_datetime(df_shark['date'], errors='coerce')
df_shark['year'] = df_shark['year'].astype(int) 

In [ ]:
df_shark.loc[df_shark['activity2'] == 'Boogie boarding', 'activity2'] = 'Body boarding'

In [ ]:
# Nueva columna diferenciancdo entre actividad de ocio y actividad no de ocio
df_shark['ocio_o_no'] = 'No ocio'
df_shark.loc[df_shark['activity2'].isin(['Surfing', 'Swimming', 'Snorkeling', 'Body boarding', 'Paddling', 'Diving', 
                                         'Kite Boarding', 'Playing', 'Rowing', 'Surf skiing', 'Paddle boarding',
                                         'Kayaking', 'Walking', 'Sitting on surfboard', 'Collecting', 'Canoeing']),
                                         'ocio_o_no'] = 'Ocio'
df_shark.loc[df_shark['activity2'].isin(['Random Activity', 'No details'])] = 'Desconocido'

In [ ]:
# Combierto zonas atacadas en puntos geolocalizables de latitud y longitud para poder hacer un mapa visual de los datos
df_shark['geolocalizacion'] = df_shark.apply(lambda row: geo_point(row, 'area', 'country'), axis=1)

In [ ]:
df_shark_2 = df_shark[~df_shark['geolocalizacion'].isna()]
df_shark.shape, df_shark_2.shape

((5738, 14), (5257, 14))

In [ ]:
df_shark_2['type'].value_counts()

type
Unprovoked      3935
Provoked         515
Invalid          398
Watercraft       287
Sea Disaster      85
Unverified        17
Questionable      15
Boat               5
Name: count, dtype: int64

In [ ]:
df_shark.iloc[-1]

date               Desconocido
year               Desconocido
type               Desconocido
country            Desconocido
area               Desconocido
location           Desconocido
activity           Desconocido
name               Desconocido
sex                Desconocido
fatal_y_n          Desconocido
species            Desconocido
activity2          Desconocido
decade             Desconocido
geolocalizacion    Desconocido
ocio_o_no          Desconocido
Name: 5737, dtype: object

In [ ]:
df_shark['lat'] = df_shark['geolocalizacion'].apply(lambda x: x[0] if x is not None else None)
df_shark['lon'] = df_shark['geolocalizacion'].apply(lambda x: x[1] if x is not None else None)

In [ ]:
df_shark['["lat", "lon"]'].dtypes

lat    object
lon    object
dtype: object

In [ ]:
df_shark[df_shark['fatal_y_n'] == 'Desconocido'] = 'UNKNOWN'
df_shark[df_shark['fatal_y_n'] == 'Surf skiing'] = 'N'
df_shark[df_shark['fatal_y_n'] == 'F'] = 'Y'
df_shark[df_shark['fatal_y_n'] == 'n'] = 'N'
df_shark[df_shark['fatal_y_n'] == 'Nq'] = 'UNKNOWN'

In [ ]:
df_shark['fatal_y_n'].unique()

array(['Y', 'N', 'UNKNOWN'], dtype=object)

In [ ]:
df_shark_2 = pd.read_csv(r'C:\Users\Vicen\Visual Code\The_Bridge\Personal_2506_dsft_thebridge\2-Analytics\EDA\EDA-SHARK-ATTACK\src\data\shark_attack_enriquecido.csv')
df_tourism = pd.read_csv(r'C:\Users\Vicen\Visual Code\The_Bridge\Personal_2506_dsft_thebridge\2-Analytics\EDA\EDA-SHARK-ATTACK\src\data\turismo_mundial.csv')
df_tourism.drop(['Unnamed: 0', '1994', '1993', '2023', '2024'], axis=1, inplace=True)
df_tourism['pais'] = df_tourism['pais'].apply(lambda x: x.upper())

In [ ]:
total_turistas_por_ano = df_tourism.iloc[:,1:].sum(axis=0).to_frame().reset_index().rename(columns={'index':'year',0: 'count_turistas'})
total_turistas_por_ano['count_turistas'] = total_turistas_por_ano['count_turistas'] / 2
total_turistas_por_ano 


,year,count_turistas
0,1995,244795350.0
1,1996,258435067.0
2,1997,270880055.0
3,1998,279808107.5
4,1999,294018094.5
5,2000,320405957.0
6,2001,319369949.0
7,2002,328511926.5
8,2003,323652399.0
9,2004,355388696.5


In [ ]:
anos_buenos = df_shark_2[(df_shark_2['year'] != 'UNKNOWN') & (df_shark_2['year'] != 'N') & (df_shark_2['year'] != 'Y')]

In [ ]:
anos_to_merge_with_tourist = anos_buenos[anos_buenos['year'].between(1995,2022)]
total_ataques_por_ano = anos_to_merge_with_tourist.groupby('year').agg('count').iloc[:, 0].to_frame().rename(columns={'Unnamed: 0': 'count_ataques'}).reset_index()

In [ ]:
totales = pd.concat([total_turistas_por_ano, total_ataques_por_ano], axis=1)
totales['porcentaje'] = (totales['count_ataques']/totales['count_turistas']) * 100
totales

,year,count_turistas,year,count_ataques,porcentaje
0,1995,244795350.0,1995.0,64,0.000026
1,1996,258435067.0,1996.0,49,0.000019
2,1997,270880055.0,1997.0,51,0.000019
3,1998,279808107.5,1998.0,59,0.000021
4,1999,294018094.5,1999.0,60,0.000020
5,2000,320405957.0,2000.0,86,0.000027
6,2001,319369949.0,2001.0,75,0.000023
7,2002,328511926.5,2002.0,83,0.000025
8,2003,323652399.0,2003.0,86,0.000027
9,2004,355388696.5,2004.0,86,0.000024


In [ ]:
porcentaje_total = (totales['count_ataques'].sum() / totales['count_turistas'].sum()) * 100
porcentaje_total

NameError: name 'totales' is not defined

: 

In [ ]:
totales.to_csv(r'C:\Users\Vicen\Visual Code\The_Bridge\Personal_2506_dsft_thebridge\2-Analytics\EDA\EDA-SHARK-ATTACK\src\data\totales.csv')

In [ ]:
df_shark.to_csv(r'C:\Users\Vicen\Visual Code\The_Bridge\Personal_2506_dsft_thebridge\2-Analytics\EDA\EDA-SHARK-ATTACK\src\data\shark_attack_enriquecido.csv')

In [ ]:
df_shark.head()

,date,year,type,country,area,location,activity,name,sex,fatal_y_n,species,activity2,decade,geolocalizacion,ocio_o_no,lat,lon
0,2023-05-13 00:00:00,2023,Unprovoked,AUSTRALIA,South Australia,Elliston,Surfing,Simon Baccanello,M,Y,White shark,Surfing,2020,"(-30.5343665, 135.6301212)",Ocio,-30.534367,135.630121
1,2023-04-29 00:00:00,2023,Unprovoked,AUSTRALIA,Western Australia,"Yallingup, Busselton",Swimming,male,M,N,1m shark,Swimming,2020,"(-25.2303005, 121.0187246)",Ocio,-25.2303,121.018725
2,2022-10-07 00:00:00,2022,Unprovoked,AUSTRALIA,Western Australia,Port Hedland,Spearfishing,Robbie Peck,M,N,Bull shark,Fishing,2020,"(-25.2303005, 121.0187246)",No ocio,-25.2303,121.018725
3,2021-10-04 00:00:00,2021,Unprovoked,USA,Florida,"Fort Pierce State Park, St. Lucie County",Surfing,Truman Van Patrick,M,N,Unknow,Surfing,2020,"(27.7567667, -81.4639835)",Ocio,27.756767,-81.463983
4,2021-10-03 00:00:00,2021,Unprovoked,USA,Florida,"Jensen Beach, Martin County",Swimming,male,M,N,Unknow,Swimming,2020,"(27.7567667, -81.4639835)",Ocio,27.756767,-81.463983
